# 植被指数计算
按区域计算筛选 *Sentinel-2* 影像后，计算植被指数并输出成果数据至 *我的数据* 中。  
植被指数计算公式参考论文：
*Zeng, Y., Hao, D., Huete, A. et al. Optical vegetation indices for monitoring terrestrial ecosystems globally. Nat Rev Earth Environ 3, 477–493 (2022). <a href='https://doi.org/10.1038/s43017-022-00298-5' target='_blank'>https://doi.org/10.1038/s43017-022-00298-5</a>*

## 初始化环境

In [1]:
import aie
aie.Authenticate()
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


## 定义矢量区域
用户可根据自主上传的矢量边界，定义检索数据的区域

In [2]:
region = aie.Geometry({"type":"MultiPolygon","coordinates":[[[[119.454352,30.746199],[119.493179,29.878117],[120.926425,29.939513],[120.902422,30.809161],[119.454352,30.746199]]]],"bbox":[119.454352,29.878117,120.926425,30.809161]})

## *Sentinel-2* 数据检索
按照区域、时间、云量检索Sentinel-2 数据，并进行镶嵌和裁剪

In [3]:
def s2_coll(startdate, enddate):
    s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
            .filterBounds(region) \
            .filterDate(startdate, enddate) \
            .filter('eo:cloud_cover<20') \
            .limit(20) \
            .select(['B8', 'B4', 'B3', 'B2'])
    merge_image = s2.mean().clip(region)
    return merge_image
    
s2_data = s2_coll('2021-04-01', '2021-08-30')

## 植被指数计算

In [4]:
def getRVI(image):
    nir = image.select(["B8"])
    red = image.select(["B4"])
    rvi = nir.subtract(red)
    return rvi

def getEVI(image):
    evi = image.expression(
        '(2.5 * (nir - red)) /(nir + 6 * red - 7.5 * blue + 1)', 
        {
        'nir' : image.select(["B8"]),
        'red' : image.select(["B4"]),
        'blue' : image.select(["B2"])
    })
    return evi

def getNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']) 
    return ndvi

def getNIRv(image):
    nir = image.select(["B8"])
    nirv = nir.multiply(image.normalizedDifference(['B8', 'B4']))
    return nirv

  
def getGCVI(image):
    nir = image.select(["B8"])
    green = image.select(["B3"])                                                 
    gcvi = nir.divide(green).subtract(aie.Image.constant(1))
    return gcvi                                                    
  
def getSAVI(image):
    nir = image.select(["B8"])
    red = image.select(["B4"])                                                
    savi = ((nir.subtract(red)).multiply(aie.Image.constant(1.5))).divide((nir.add(red)).add(aie.Image.constant(0.5)))
    return savi 

def getNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']) 
    return ndwi
    

## 数据可视化
计算植被指数，并进行数据地图可视化展示

In [5]:
ndvi = getNDVI(s2_data)
nirv = getNIRv(s2_data)

ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': [
        '#2B83BA', '#ABDDA4', '#FFFFBF', '#FDAE61', '#D7191C'
    ]
}

nirv_vis = {
    'min': -1000,
    'max': 1000,
    'palette': [
        '#2B83BA', '#ABDDA4', '#FFFFBF', '#FDAE61', '#D7191C'
    ]
}

map = aie.Map(
    center=region.getCenter(),            #中心点
    height=800,   
    zoom=6                               #地图层级
)

map.addLayer(
    ndvi,  
    ndvi_vis,
    'NDVI',
    bounds=region.getBounds()
)

map.addLayer(
    nirv,  
    nirv_vis,
    'NIRv',
    bounds=region.getBounds()
)

map


Map(center=[30.343639, 120.1903885], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## 影像输出
将影像输出至 *我的数据* 中

In [6]:
task = aie.Export.image.toAsset(ndvi, 'NDVI', 10)
task.start()
task = aie.Export.image.toAsset(nirv, 'NIRv', 10)
task.start()